# Korean LLM (Large Language Model) fine-tuning on Local environment (Debugging)
---

- 허깅페이스 인증 정보 설정: `huggingface-cli login`
    - https://huggingface.co/join
    - https://huggingface.co/settings/tokens
    

## Overview 

본격적으로 SageMaker 훈련 인스턴스로 훈련을 수행하기 전에 SageMaker Notebook / SageMaker Studio / SageMaker Studio Lab 에서 샘플 데이터로 디버깅을 수행합니다.
물론 온프레미스 환경에서 디버깅을 수행할 수 있다면, 기존 환경과 동일하게 디버깅을 수행하면 됩니다.


In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import torch
import transformers
from datasets import load_dataset, load_from_disk
from transformers import GPTNeoXForCausalLM, GPTNeoXTokenizerFast

sys.path.append('./utils')
sys.path.append('./templates')

In [2]:
%store -r bucket_prefix dataset_prefix s3_data_path dataset_prefix_all

In [3]:
try:
    dataset_prefix
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] 1번 모듈 노트북을 다시 실행해 주세요.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [4]:
lm_dataset = load_from_disk(dataset_prefix)

In [5]:
#lm_dataset = load_from_disk(dataset_prefix_all)

In [6]:
lm_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 50
})

In [7]:
dataset_prefix

'chunk-train'

<br>

## 1. Load Model
---
한정된 GPU 메모리 안에 LLM 모델을 로드하는 것은 매우 어렵습니다. 예컨대 20B의 모델을 로드하려면 fp32 기준으로 80GB 이상의 메모리가 필요하고 fp16 기준으로도 40GB 이상의 GPU 메모리가 필요하며, 파인 튜닝을 수행하는 경우는 이보다 더욱 많은 GPU 메모리가 필요합니다. 이런 경우 4비트 양자화와 LoRA를 사용하면 범용적으로 사용하고 있는 16GB 및 24GB GPU 메모리로도 파인 튜닝이 가능합니다. 현 기준으로는 4비트 양자화를 지원하는 QLoRA 기법이 가장 널리 사용되고 있으며 bitsandbytes를 사용하여 QLoRA를 쉽게 적용할 수 있습니다. QLoRA는 양자화된 파라미터의 분포 범위를 정규 분포 내로 억제하여 정밀도의 저하를 방지하는 4비트 NormalFloat 양자화 양자화를 적용하는 정수에 대해서도 양자화를 적용하는 이중 양자화, 그리고 optimizer state 등의 데이터를 CPU 메모리에 저장하는 페이징 기법을 적용하여 GPU 메모리 사용량을 억제합니다. QLoRA에 대한 자세한 내용은 논문 (https://arxiv.org/pdf/2305.14314.pdf) 을 참조하기 바랍니다.

### Create a bitsandbytes configuration

In [8]:
from transformers import BitsAndBytesConfig
quant_4bit = True
quant_8bit = False

if quant_4bit:
    nf4_config = BitsAndBytesConfig(
       load_in_4bit=True,
       bnb_4bit_quant_type="nf4",
       bnb_4bit_use_double_quant=True,
       bnb_4bit_compute_dtype=torch.bfloat16
)
else:
    nf4_config = None

In [9]:
torch.cuda.empty_cache()

In [10]:
import os
from pathlib import Path
from huggingface_hub import snapshot_download

HF_MODEL_ID = "nlpai-lab/kullm-polyglot-12.8b-v2"

# create model dir
model_name = HF_MODEL_ID.split("/")[-1].replace('.', '-')
model_tar_dir = Path(f"/home/ec2-user/SageMaker/models/{model_name}")

In [11]:
device_map = "auto"

tokenizer = GPTNeoXTokenizerFast.from_pretrained(HF_MODEL_ID)

model = GPTNeoXForCausalLM.from_pretrained(
    model_tar_dir,
    load_in_8bit=True if quant_8bit else False,
    torch_dtype=torch.float16,
    device_map=device_map,
    #cache_dir=cache_dir,
    quantization_config=nf4_config,
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'GPTNeoXTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Create LoRA config
LoRA 설정에 대한 자세한 내용은 아래를 참조해 주세요.
- https://huggingface.co/docs/peft/conceptual_guides/lora

In [12]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
)

model = prepare_model_for_kbit_training(model)

lora_r  = 8
lora_alpha = 32
lora_dropout = 0.05
lora_target_modules = ["query_key_value", "xxx"]
    
config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

In [13]:
model.print_trainable_parameters()

trainable params: 6,553,600 || all params: 12,900,157,440 || trainable%: 0.05080248074863806


<br>

## 2. Training
---
### Setting Hyperparameters

In [15]:
train_data = lm_dataset
val_data = None
num_epochs = 3
batch_size = 2

learning_rate = 3e-5
gradient_accumulation_steps = 2
val_set_size = 0
output_dir = 'output'
world_size = 1
ddp = world_size != 1
group_by_length = False

In [16]:
train_args = transformers.TrainingArguments(
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    warmup_steps=100,
    num_train_epochs=num_epochs,
    learning_rate=learning_rate,
    bf16=True,
    logging_steps=2,
    optim="paged_adamw_8bit",
    evaluation_strategy="steps" if val_set_size > 0 else "no",
    save_strategy="steps",
    eval_steps=200 if val_set_size > 0 else None,
    save_steps=10,
    output_dir=output_dir,
    load_best_model_at_end=True if val_set_size > 0 else False,
    ddp_find_unused_parameters=False if ddp else None,
    report_to="none",
    group_by_length=group_by_length,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=train_args,
    data_collator=transformers.DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)

In [17]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                

### Start Training

In [18]:
model.config.use_cache = False

# old_state_dict = model.state_dict
# model.state_dict = (lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())).__get__(
#     model, type(model)
# )

if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

train_result = trainer.train()

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
2,1.970600
4,1.837500
6,1.886500
8,1.978900
10,1.923500
12,1.842900
14,1.934400
16,1.957300
18,1.807000
20,1.895300


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/ec2-user/anaconda3/e

### Check metrics

In [19]:
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
#trainer.save_metrics("train", metrics)

***** train metrics *****
  epoch                    =       2.88
  total_flos               = 15753753GF
  train_loss               =     1.9033
  train_runtime            = 0:10:06.24
  train_samples_per_second =      0.247
  train_steps_per_second   =      0.059


### Save fine-tuned model

In [20]:
trainer.model.save_pretrained(output_dir)

In [21]:
tokenizer.save_pretrained(output_dir)

('output/tokenizer_config.json',
 'output/special_tokens_map.json',
 'output/tokenizer.json')

In [22]:
%store output_dir

Stored 'output_dir' (str)


In [23]:
# Free memory for merging weights
#del model
#del trainer
#torch.cuda.empty_cache()

In [24]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [25]:
model.generate(**tokenizer("### 질문: 안녕 나는 빡빡이 아저씨야", return_tensors='pt', return_token_type_ids=False))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[    6,     6,     6,  2438,    29,  5565,   462,   272, 26343,   270,
          6976,   489,    17,   503,   293,  7735,   272,   388,   296,  3133]])

In [26]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:", 
            return_tensors='pt', 
            return_token_type_ids=False
        ), 
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [27]:
gen('프로그래밍을 잘 하기 위한 세 가지 방법은?')

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 프로그래밍을 잘 하기 위한 세 가지 방법은?

### 답변:
세 가지 방법의 예에는 다음이 포함됩니다:1. 알고리즘을 구체적으로 작성하기: 문제를 식별하고, 필요한 것을 정확히 계산하고, 실행 가능한 구현을 작성하여 알고리즘을 구체적이고 이해하기 쉽게 만듭니다.2. 컴퓨터와 더 잘 작업하기: 컴퓨터가 내가 생각하는 것을 이해할 수 있도록 컴퓨터의 언어로 나의 생각을 설명합니다. 이는 코드, 텍스트 설명 또는 기타 구두 표현을 통해 이루어질 수 있습니다.3. 반복하기: 알고리즘을 처음부터 다시 작성합니다. 문제에 대해 새로운 아이디어를 더하고, 알고리즘을 수정하고, 구현을 테스트하여 작업에 대한 이해를 보장합니다.<|endoftext|>


### 2_local-infer-debug-lora 과정을 위해 allocated CUDA memory를 Release합니다. 

In [28]:
# Free memory for merging weights
del model
del trainer
torch.cuda.empty_cache()